In [12]:
import numpy as np 
from tqdm import tqdm
import cv2
import os
import shutil
import itertools
import imutils
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools

from keras import layers, models
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.layers.core import Activation, Dense, Flatten, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras import backend as K

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications import VGG19
from keras.applications.resnet50 import ResNet50
from keras.applications import ResNet101
from keras.applications import DenseNet201, DenseNet169, DenseNet121
from keras.applications import InceptionV3
from keras.applications import InceptionResNetV2
from keras.applications import Xception
from keras.applications import MobileNetV2
from keras.applications import MobileNet
from keras import layers
from keras.models import Model, Sequential
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping
import pandas as pd

from sklearn.svm import SVC

init_notebook_mode(connected=True)
RANDOM_SEED = 123

In [13]:
def load_data(dir_path, img_size=(100, 100)):
    X = []
    y = []
    i = 0
    labels = dict()
    for path in tqdm(sorted(os.listdir(dir_path))):
        if not path.startswith('.'):
            labels[i] = path
            for file in os.listdir(dir_path + path):
                if not file.startswith('.'):
                    img = cv2.imread(dir_path + path + '/' + file)
                    X.append(img)
                    y.append(i)
            i += 1
    X = np.array(X)
    y = np.array(y)
    print('{} images loaded from {} directory.'.format(len(X), dir_path))
    return X, y, labels
    

In [14]:
TRAIN_DIR = 'TRAIN/'
TEST_DIR = 'TEST/'
VAL_DIR = 'VAL/'
IMG_SIZE = (224, 224)

X_train, y_train, labels = load_data(TRAIN_DIR, IMG_SIZE)
X_test, y_test, _ = load_data(TEST_DIR, IMG_SIZE)
X_val, y_val, _ = load_data(VAL_DIR, IMG_SIZE)

100%|███████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  4.05it/s]
C:\Users\zeyne\miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:15: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray

  0%|                                                                           | 0/2 [00:00<?, ?it/s]

242 images loaded from TRAIN/ directory.
19 images loaded from TEST/ directory.


100%|███████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 11.39it/s]

92 images loaded from VAL/ directory.


In [15]:
def preprocess_imgs(set_name, img_size):
    
    # Resize and apply VGG-15 preprocessing
    set_new = []
    for img in set_name:
        img = cv2.resize(
            img,
            dsize=img_size,
            interpolation=cv2.INTER_CUBIC
        )
        set_new.append(preprocess_input(img))
    return np.array(set_new)

In [16]:
X_train_prep = preprocess_imgs(set_name=X_train, img_size=IMG_SIZE)
X_test_prep = preprocess_imgs(set_name=X_test, img_size=IMG_SIZE)
X_val_prep = preprocess_imgs(set_name=X_val, img_size=IMG_SIZE)

# CNN Pre-trained Models

## Data Augmentation

In [17]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    brightness_range=[0.5, 1.5],
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=preprocess_input
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    color_mode='rgb',
    target_size=IMG_SIZE,
    batch_size=8,
    class_mode='binary',
    seed=RANDOM_SEED
)

validation_generator = test_datagen.flow_from_directory(
    VAL_DIR,
    color_mode='rgb',
    target_size=IMG_SIZE,
    batch_size=8,
    class_mode='binary',
    seed=RANDOM_SEED
)

Found 242 images belonging to 2 classes.
Found 92 images belonging to 2 classes.


In [18]:
nets_evaluation_df = pd.DataFrame(columns=['Training Accuracy', 'Validation Accuracy', 'Test Accuracy', 'F1 Measure', 'Kappa', 'ROC Area'])
EPOCHS = 30
NUM_CLASSES = 1
trained_nets = {VGG16 : 'VGG16', VGG19 : 'VGG19',  ResNet50 : 'ResNet50', ResNet101 : 'ResNet101', DenseNet201 : 'DenseNet201',
               DenseNet169 : 'DenseNet169', DenseNet121 : 'DenseNet121', InceptionV3 : 'InceptionV3', InceptionResNetV2 : 'InceptionResNetV2',
               Xception : 'Xception', MobileNetV2 : 'MobileNetV2', MobileNet : 'MobileNet'}
clf = SVC(kernel='linear')

In [19]:
measures = {}

In [20]:
for net in trained_nets.keys():
    # loading base model
    base_model = net(
    weights='imagenet',
    include_top=False,
    input_shape=IMG_SIZE + (3,))
    
    

    model= Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(NUM_CLASSES, activation='sigmoid'))
    
    model.layers[0].trainable = False

    model.compile(
        loss='binary_crossentropy',
        optimizer=RMSprop(lr=1e-4),
        metrics=['accuracy'])
    
    feature_extractor = Model(model.inputs, model.layers[-3].output)
    
    history = model.fit(
                    train_generator, steps_per_epoch=len(train_generator),
                    epochs=EPOCHS, validation_data=validation_generator,
                    validation_steps=len(validation_generator))
    
    features = feature_extractor.predict(X_train_prep)
    
    clf.fit(features, y_train)
    
    predictions = clf.predict(X_test_prep)
    predictions = [1 if x>0.5 else 0 for x in predictions]
    
    test_accuracy = accuracy_score(y_test, predictions)
    
    measures[trained_nets[net]] = test_accuracy

Epoch 1/30
31/31 [==============================] - 9s 288ms/step - loss: 4.0806 - accuracy: 0.6694 - val_loss: 2.0639 - val_accuracy: 0.9130
Epoch 2/30
31/31 [==============================] - 3s 108ms/step - loss: 2.6526 - accuracy: 0.7273 - val_loss: 5.5225e-09 - val_accuracy: 0.7935
Epoch 3/30
31/31 [==============================] - 3s 108ms/step - loss: 1.8311 - accuracy: 0.7975 - val_loss: 1.9022e-05 - val_accuracy: 0.9565
Epoch 4/30
31/31 [==============================] - 3s 108ms/step - loss: 1.8984 - accuracy: 0.7769 - val_loss: 3.0695e-07 - val_accuracy: 0.9674
Epoch 5/30
31/31 [==============================] - 3s 108ms/step - loss: 1.8776 - accuracy: 0.7645 - val_loss: 0.0141 - val_accuracy: 0.9348
Epoch 6/30
31/31 [==============================] - 3s 108ms/step - loss: 1.0324 - accuracy: 0.8223 - val_loss: 0.0269 - val_accuracy: 0.9565
Epoch 7/30
31/31 [==============================] - 3s 109ms/step - loss: 1.0267 - accuracy: 0.8058 - val_loss: 2.3609e-04 - val_accurac

ValueError: Found array with dim 4. Estimator expected <= 2.